# Single-Stock and Cross-Sectional Feature Computation Visualization

This notebook demonstrates the new centralized feature computation functions:
- `compute_single_stock_features()`: Features computed for individual stocks
- `compute_cross_sectional_features()`: Features requiring comparisons across multiple stocks

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Add project src to path
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import the new centralized feature functions
from src.features import compute_single_stock_features, compute_cross_sectional_features
from src.data.loader import load_stock_universe, load_etf_universe
from src.features.assemble import assemble_indicators_from_wide

print("✅ Successfully imported feature computation functions")
print(f"📁 Working directory: {os.getcwd()}")

✅ Successfully imported feature computation functions
📁 Working directory: /mnt/a61cc0e8-1b32-4574-a771-4ad77e8faab6/conda/technical_dashboard/notebooks


## 1. Load Stock and ETF Data

Load stocks AND ETFs (including SPY) - ETFs are needed for cross-sectional comparisons.

In [2]:
print("📊 Loading stock data...")

# Load stock data (limit to 10 symbols for quick demonstration)
stocks, sectors = load_stock_universe(
    max_symbols=10,
    update=False,
    include_sectors=True
)

print(f"✅ Loaded {len(stocks)} stock price series")
print(f"   Symbols: {list(stocks['AdjClose'].columns[:10])}")
print(f"   Date range: {stocks['AdjClose'].index.min()} to {stocks['AdjClose'].index.max()}")
print(f"   Sectors: {len(sectors)} symbols mapped to sectors")

📊 Loading stock data...
💾 stocks: loading cache /mnt/a61cc0e8-1b32-4574-a771-4ad77e8faab6/conda/technical_dashboard/cache/stock_data_universe.parquet
📊 Applied max_symbols limit: reduced to 10 symbols
✅ Loaded 6 stock price series
   Symbols: ['A', 'AA', 'AACB', 'AACI', 'AACT', 'AAL', 'AAMI', 'AAOI', 'AAON', 'AAP']
   Date range: 2020-09-21 00:00:00 to 2025-09-19 00:00:00
   Sectors: 10 symbols mapped to sectors


In [ ]:
print("\n📊 Loading ETF data (including SPY for cross-sectional features)...")

# Load ETFs - these are needed for cross-sectional comparisons
etfs = load_etf_universe(
    etf_symbols=['SPY', 'XLK', 'XLF', 'XLV', 'XLP', 'XLE', 'XLI', 'XLY', 'XLU', 'XLB'],
    update=False
)

print(f"✅ Loaded {len(etfs)} ETF price series")
if 'AdjClose' in etfs:
    print(f"   ETF Symbols: {list(etfs['AdjClose'].columns)}")
    print(f"   SPY included: {'SPY' in etfs['AdjClose'].columns}")

In [ ]:
# Combine stocks and ETFs
print("\n🔧 Combining stocks and ETFs...")

combined_data = {}
for metric in stocks.keys():
    stock_df = stocks.get(metric, pd.DataFrame())
    etf_df = etfs.get(metric, pd.DataFrame())
    combined_data[metric] = pd.concat([stock_df, etf_df], axis=1).sort_index()

print(f"✅ Combined data has {len(combined_data)} metrics")
if 'AdjClose' in combined_data:
    print(f"   Total symbols: {len(combined_data['AdjClose'].columns)}")
    print(f"   Includes SPY: {'SPY' in combined_data['AdjClose'].columns}")

In [ ]:
# Assemble indicators from wide format
print("\n🔧 Assembling indicator DataFrames...")
indicators_by_symbol = assemble_indicators_from_wide(combined_data, adjust_ohlc_with_factor=True)

print(f"✅ Assembled {len(indicators_by_symbol)} symbol DataFrames")
print(f"   Includes SPY: {'SPY' in indicators_by_symbol}")
sample_sym = list(indicators_by_symbol.keys())[0]
print(f"   Example columns for {sample_sym}: {list(indicators_by_symbol[sample_sym].columns)}")

## 2. Compute Single-Stock Features

Apply the `compute_single_stock_features()` function to each stock/ETF independently.

In [ ]:
print("🔨 Computing single-stock features...")

# Store results before and after for comparison
original_columns = list(indicators_by_symbol[sample_sym].columns)

# Apply single-stock features to each symbol (including ETFs)
for symbol, df in indicators_by_symbol.items():
    indicators_by_symbol[symbol] = compute_single_stock_features(df)

# Show what features were added
new_columns = list(indicators_by_symbol[sample_sym].columns)
added_features = set(new_columns) - set(original_columns)

print(f"✅ Single-stock features computed for {len(indicators_by_symbol)} symbols")
print(f"   Original columns: {len(original_columns)}")
print(f"   New columns: {len(new_columns)}")
print(f"   Features added: {len(added_features)}")
print(f"\n📋 Sample of added features:")
for i, feat in enumerate(sorted(added_features)[:15]):
    print(f"   {i+1}. {feat}")

## 3. Visualize Single-Stock Features

Examine the single-stock features for one stock symbol (not SPY).

In [ ]:
# Select a stock symbol to visualize (not SPY)
stock_symbols = [s for s in indicators_by_symbol.keys() if s != 'SPY' and s not in ['XLK', 'XLF', 'XLV', 'XLP', 'XLE', 'XLI', 'XLY', 'XLU', 'XLB']]
viz_symbol = stock_symbols[0] if stock_symbols else list(indicators_by_symbol.keys())[0]
df = indicators_by_symbol[viz_symbol].copy()

# Filter to recent data for clearer visualization
df_recent = df.tail(252)  # Last year of trading days

print(f"📊 Visualizing single-stock features for {viz_symbol}")
print(f"   Period: {df_recent.index.min()} to {df_recent.index.max()}")

# Create comprehensive visualization
fig, axes = plt.subplots(5, 1, figsize=(16, 20))
fig.suptitle(f'Single-Stock Features for {viz_symbol}', fontsize=16, fontweight='bold')

# 1. Price and Moving Averages
ax1 = axes[0]
ax1.plot(df_recent.index, df_recent['adjclose'], label='Adj Close', color='black', linewidth=2)
ma_cols = [col for col in df_recent.columns if col.startswith('ma_')]
colors = plt.cm.viridis(np.linspace(0, 1, len(ma_cols)))
for i, col in enumerate(sorted(ma_cols)[:5]):  # Show first 5 MAs
    ax1.plot(df_recent.index, df_recent[col], label=col, alpha=0.7, color=colors[i])
ax1.set_ylabel('Price ($)', fontweight='bold')
ax1.set_title('Price and Moving Averages', fontweight='bold')
ax1.legend(loc='best', ncol=2)
ax1.grid(True, alpha=0.3)

# 2. Volatility Regime
ax2 = axes[1]
if 'vol_regime' in df_recent.columns and 'vol_regime_ema10' in df_recent.columns:
    ax2.plot(df_recent.index, df_recent['vol_regime'], label='Vol Regime', alpha=0.5)
    ax2.plot(df_recent.index, df_recent['vol_regime_ema10'], label='Vol Regime EMA10', linewidth=2)
    ax2.axhline(y=0, color='red', linestyle='--', alpha=0.7)
    ax2.fill_between(df_recent.index, df_recent['vol_regime_ema10'], 0, 
                     where=(df_recent['vol_regime_ema10'] > 0), alpha=0.3, color='red', label='High Vol')
    ax2.fill_between(df_recent.index, df_recent['vol_regime_ema10'], 0,
                     where=(df_recent['vol_regime_ema10'] < 0), alpha=0.3, color='green', label='Low Vol')
ax2.set_ylabel('Vol Regime', fontweight='bold')
ax2.set_title('Volatility Regime (Short/Long Vol Ratio)', fontweight='bold')
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)

# 3. RSI Indicators
ax3 = axes[2]
rsi_cols = [col for col in df_recent.columns if col.startswith('rsi_')]
colors_rsi = ['green', 'orange', 'red']
for i, col in enumerate(sorted(rsi_cols)[:3]):
    ax3.plot(df_recent.index, df_recent[col], label=col.upper(), color=colors_rsi[i], linewidth=2)
ax3.axhline(y=70, color='red', linestyle='--', alpha=0.7, label='Overbought')
ax3.axhline(y=30, color='green', linestyle='--', alpha=0.7, label='Oversold')
ax3.axhline(y=50, color='gray', linestyle='-', alpha=0.5)
ax3.set_ylim(0, 100)
ax3.set_ylabel('RSI', fontweight='bold')
ax3.set_title('Relative Strength Index (Multiple Periods)', fontweight='bold')
ax3.legend(loc='best', ncol=2)
ax3.grid(True, alpha=0.3)

# 4. MACD Features
ax4 = axes[3]
if 'macd_histogram' in df_recent.columns:
    ax4.bar(df_recent.index, df_recent['macd_histogram'], alpha=0.6, color='blue', width=1, label='MACD Histogram')
    ax4.axhline(y=0, color='red', linestyle='--', alpha=0.7)
if 'macd_hist_deriv_ema3' in df_recent.columns:
    ax4_twin = ax4.twinx()
    ax4_twin.plot(df_recent.index, df_recent['macd_hist_deriv_ema3'], color='purple', linewidth=2, label='MACD Deriv EMA3')
    ax4_twin.set_ylabel('MACD Derivative', color='purple', fontweight='bold')
    ax4_twin.tick_params(axis='y', labelcolor='purple')
    ax4_twin.legend(loc='upper right')
ax4.set_ylabel('MACD Histogram', color='blue', fontweight='bold')
ax4.set_title('MACD Histogram and Derivative', fontweight='bold')
ax4.legend(loc='upper left')
ax4.grid(True, alpha=0.3)

# 5. Volume Features
ax5 = axes[4]
if 'volume' in df_recent.columns:
    ax5.bar(df_recent.index, df_recent['volume'], alpha=0.3, color='gray', width=1, label='Volume')
vol_shock_cols = [col for col in df_recent.columns if 'volshock' in col]
if vol_shock_cols:
    ax5_twin = ax5.twinx()
    # Pick a couple volume shock features
    for col in vol_shock_cols[:2]:
        ax5_twin.plot(df_recent.index, df_recent[col], label=col, linewidth=2)
    ax5_twin.set_ylabel('Volume Shock', fontweight='bold')
    ax5_twin.legend(loc='upper right')
ax5.set_ylabel('Volume', fontweight='bold')
ax5.set_title('Volume and Volume Shock Features', fontweight='bold')
ax5.legend(loc='upper left')
ax5.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📈 Visualization complete for {viz_symbol}")

## 4. Compute Cross-Sectional Features

Now apply the `compute_cross_sectional_features()` function to add features that require comparing across multiple stocks.

**Note:** SPY must be in the indicators_by_symbol dict for relative strength features to work!

In [ ]:
print("🔨 Building enhanced sector/subsector mappings...")

# Import the enhanced mapping builder
from src.features.sector_mapping import build_enhanced_sector_mappings

# Path to universe CSV (contains sector and industry metadata)
universe_csv = os.path.join(project_root, 'data', 'universe.csv')

if os.path.exists(universe_csv):
    # Build enhanced mappings using stock and ETF data
    enhanced_mappings = build_enhanced_sector_mappings(
        universe_csv=universe_csv,
        stock_data={sym: df for sym, df in indicators_by_symbol.items() if sym in stock_tickers},
        etf_data={sym: df for sym, df in indicators_by_symbol.items() if sym in etf_list},
        base_sectors=filtered_sectors
    )
    
    print(f"✅ Built enhanced mappings for {len(enhanced_mappings)} symbols")
    
    # Show example mapping for visualization symbol
    if viz_symbol in enhanced_mappings:
        print(f"\n📋 Enhanced mapping for {viz_symbol}:")
        for key, value in enhanced_mappings[viz_symbol].items():
            print(f"   {key}: {value}")
else:
    print(f"⚠️  Universe CSV not found at {universe_csv}")
    print("   Proceeding without enhanced mappings (no subsector features)")
    enhanced_mappings = None

In [ ]:
print("🔨 Computing cross-sectional features...")
print(f"   SPY available: {'SPY' in indicators_by_symbol}")

# Store column count before cross-sectional features
pre_cs_columns = list(indicators_by_symbol[viz_symbol].columns)

# Prepare sector mapping (use only symbols we have)
available_symbols = set(indicators_by_symbol.keys())
filtered_sectors = {sym: sec for sym, sec in sectors.items() if sym in available_symbols}

# Simple sector to ETF mapping
sector_to_etf = {
    "technology services": "XLK",
    "electronic technology": "XLK",
    "finance": "XLF",
    "health technology": "XLV",
    "consumer non-durables": "XLP",
    "energy minerals": "XLE",
}

# Get list of actual stock tickers (not ETFs) for breadth calculation
etf_list = ['SPY', 'XLK', 'XLF', 'XLV', 'XLP', 'XLE', 'XLI', 'XLY', 'XLU', 'XLB', 'RSP']
stock_tickers = [s for s in available_symbols if s not in etf_list]

print(f"   Stock symbols for breadth: {len(stock_tickers)}")
print(f"   ETFs available: {[e for e in etf_list if e in available_symbols]}")

# Compute cross-sectional features
compute_cross_sectional_features(
    indicators_by_symbol,
    sectors=filtered_sectors,
    sector_to_etf=sector_to_etf,
    market_symbol="SPY",
    sp500_tickers=stock_tickers,
    enhanced_mappings=None,
    beta_win=60,
    alpha_windows=(20, 60, 120),
    alpha_ema_span=10,
    xsec_lookbacks=(5, 20, 60),
    price_col="adjclose"
)

# Show what cross-sectional features were added
post_cs_columns = list(indicators_by_symbol[viz_symbol].columns)
cs_added_features = set(post_cs_columns) - set(pre_cs_columns)

print(f"\n✅ Cross-sectional features computed for {len(indicators_by_symbol)} symbols")
print(f"   Columns before: {len(pre_cs_columns)}")
print(f"   Columns after: {len(post_cs_columns)}")
print(f"   Cross-sectional features added: {len(cs_added_features)}")
print(f"\n📋 Cross-sectional features added:")
for i, feat in enumerate(sorted(cs_added_features)[:20]):
    print(f"   {i+1}. {feat}")

## 5. Visualize Cross-Sectional Features

Examine how cross-sectional features compare the stock to the market.

In [ ]:
# Visualize cross-sectional features for the same symbol
df_with_cs = indicators_by_symbol[viz_symbol].copy()
df_cs_recent = df_with_cs.tail(252)

print(f"📊 Visualizing cross-sectional features for {viz_symbol}")

# Create more comprehensive visualization
fig, axes = plt.subplots(6, 1, figsize=(16, 24))
fig.suptitle(f'Cross-Sectional Features for {viz_symbol}', fontsize=16, fontweight='bold')

# 1. Volatility Regime - Individual vs Cross-Sectional
ax1 = axes[0]
if 'vol_regime' in df_cs_recent.columns:
    ax1.plot(df_cs_recent.index, df_cs_recent['vol_regime'], label='Vol Regime (Individual)', linewidth=2)
if 'vol_regime_cs_median' in df_cs_recent.columns:
    ax1.plot(df_cs_recent.index, df_cs_recent['vol_regime_cs_median'], label='Vol Regime (Market Median)', linewidth=2, linestyle='--')
if 'vol_regime_rel' in df_cs_recent.columns:
    ax1.plot(df_cs_recent.index, df_cs_recent['vol_regime_rel'], label='Vol Regime (Relative)', linewidth=2, alpha=0.7)
    ax1.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax1.set_ylabel('Volatility Regime', fontweight='bold')
ax1.set_title('Volatility Regime: Individual vs Market', fontweight='bold')
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# 2. Relative Strength vs SPY (Market)
ax2 = axes[1]
rs_spy_cols = [col for col in df_cs_recent.columns if 'rel_strength_spy' in col and 'norm' in col]
if rs_spy_cols:
    ax2.plot(df_cs_recent.index, df_cs_recent['rel_strength_spy_norm'], label='RS vs SPY (norm)', linewidth=2, color='blue')
    ax2.axhline(y=0, color='red', linestyle='--', alpha=0.7)
    # Add slope on secondary axis
    if 'rel_strength_spy_slope20' in df_cs_recent.columns:
        ax2_twin = ax2.twinx()
        ax2_twin.plot(df_cs_recent.index, df_cs_recent['rel_strength_spy_slope20'], 
                     label='RS Slope (20d)', linewidth=1.5, color='orange', alpha=0.7)
        ax2_twin.set_ylabel('Slope (log returns/day)', color='orange', fontweight='bold')
        ax2_twin.tick_params(axis='y', labelcolor='orange')
        ax2_twin.axhline(y=0, color='orange', linestyle=':', alpha=0.5)
        ax2_twin.legend(loc='upper right')
    ax2.set_ylabel('Normalized RS', fontweight='bold')
    ax2.set_title('Relative Strength vs SPY (Market)', fontweight='bold')
    ax2.legend(loc='upper left')
else:
    ax2.text(0.5, 0.5, 'No SPY relative strength features\n(SPY may not be in indicators_by_symbol)', 
             ha='center', va='center', transform=ax2.transAxes, fontsize=12, color='red')
ax2.grid(True, alpha=0.3)

# 3. Relative Strength vs Sector ETF
ax3 = axes[2]
rs_sector_cols = [col for col in df_cs_recent.columns if 'rel_strength_sector' in col and 'norm' in col and 'subsector' not in col]
if rs_sector_cols:
    ax3.plot(df_cs_recent.index, df_cs_recent['rel_strength_sector_norm'], label='RS vs Sector (norm)', linewidth=2, color='green')
    ax3.axhline(y=0, color='red', linestyle='--', alpha=0.7)
    # Add slope on secondary axis
    if 'rel_strength_sector_slope20' in df_cs_recent.columns:
        ax3_twin = ax3.twinx()
        ax3_twin.plot(df_cs_recent.index, df_cs_recent['rel_strength_sector_slope20'], 
                     label='Sector RS Slope (20d)', linewidth=1.5, color='purple', alpha=0.7)
        ax3_twin.set_ylabel('Slope (log returns/day)', color='purple', fontweight='bold')
        ax3_twin.tick_params(axis='y', labelcolor='purple')
        ax3_twin.axhline(y=0, color='purple', linestyle=':', alpha=0.5)
        ax3_twin.legend(loc='upper right')
    ax3.set_ylabel('Normalized RS', fontweight='bold')
    ax3.set_title('Relative Strength vs Sector ETF', fontweight='bold')
    ax3.legend(loc='upper left')
else:
    ax3.text(0.5, 0.5, 'No sector relative strength features\n(Sector ETF may not be available)', 
             ha='center', va='center', transform=ax3.transAxes, fontsize=12, color='gray')
ax3.grid(True, alpha=0.3)

# 4. Relative Strength vs Subsector ETF (if enhanced mappings used)
ax4 = axes[3]
rs_subsector_cols = [col for col in df_cs_recent.columns if 'rel_strength_subsector' in col and 'norm' in col]
if rs_subsector_cols:
    ax4.plot(df_cs_recent.index, df_cs_recent['rel_strength_subsector_norm'], label='RS vs Subsector (norm)', linewidth=2, color='teal')
    ax4.axhline(y=0, color='red', linestyle='--', alpha=0.7)
    # Add slope on secondary axis
    if 'rel_strength_subsector_slope20' in df_cs_recent.columns:
        ax4_twin = ax4.twinx()
        ax4_twin.plot(df_cs_recent.index, df_cs_recent['rel_strength_subsector_slope20'], 
                     label='Subsector RS Slope (20d)', linewidth=1.5, color='brown', alpha=0.7)
        ax4_twin.set_ylabel('Slope (log returns/day)', color='brown', fontweight='bold')
        ax4_twin.tick_params(axis='y', labelcolor='brown')
        ax4_twin.axhline(y=0, color='brown', linestyle=':', alpha=0.5)
        ax4_twin.legend(loc='upper right')
    ax4.set_ylabel('Normalized RS', fontweight='bold')
    ax4.set_title('Relative Strength vs Subsector ETF', fontweight='bold')
    ax4.legend(loc='upper left')
else:
    ax4.text(0.5, 0.5, 'No subsector relative strength features\n(Enhanced mappings not provided)', 
             ha='center', va='center', transform=ax4.transAxes, fontsize=12, color='gray')
ax4.grid(True, alpha=0.3)

# 5. Alpha Features (if available)
ax5 = axes[4]
alpha_cols = [col for col in df_cs_recent.columns if 'alpha_' in col]
if alpha_cols:
    for col in sorted(alpha_cols)[:3]:  # Show first 3 alpha features
        ax5.plot(df_cs_recent.index, df_cs_recent[col], label=col, linewidth=2)
    ax5.axhline(y=0, color='red', linestyle='--', alpha=0.7)
    ax5.set_ylabel('Alpha', fontweight='bold')
    ax5.set_title('Alpha vs Market/Sector', fontweight='bold')
    ax5.legend(loc='best')
else:
    ax5.text(0.5, 0.5, 'No alpha features available', ha='center', va='center', transform=ax5.transAxes)
ax5.grid(True, alpha=0.3)

# 6. Cross-Sectional Momentum Z-Scores
ax6 = axes[5]
xmom_cols = [col for col in df_cs_recent.columns if col.startswith('xsec_mom_')]
if xmom_cols:
    for col in sorted(xmom_cols)[:3]:  # Show first 3 momentum features
        ax6.plot(df_cs_recent.index, df_cs_recent[col], label=col, linewidth=2)
    ax6.axhline(y=0, color='gray', linestyle='--', alpha=0.7, label='Median')
    ax6.axhline(y=1.0, color='green', linestyle='--', alpha=0.5, label='+1 std')
    ax6.axhline(y=-1.0, color='red', linestyle='--', alpha=0.5, label='-1 std')
    ax6.set_ylabel('Z-Score', fontweight='bold')
    ax6.set_ylim(-3, 3)
    ax6.set_title('Cross-Sectional Momentum Z-Scores', fontweight='bold')
    ax6.legend(loc='best')
else:
    ax6.text(0.5, 0.5, 'No cross-sectional momentum features', ha='center', va='center', transform=ax6.transAxes)
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📈 Cross-sectional visualization complete for {viz_symbol}")

## 6. Feature Summary Statistics

Examine the distribution of features across all stocks.

In [ ]:
print("📊 Feature Summary Statistics Across All Symbols")
print("=" * 60)

# Collect all feature data (stocks only, not ETFs)
all_data = []
for symbol, df in indicators_by_symbol.items():
    if symbol not in etf_list:  # Only include stocks
        df_copy = df.copy()
        df_copy['symbol'] = symbol
        all_data.append(df_copy)

if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)

    print(f"\n📈 Combined Dataset (stocks only):")
    print(f"   Total rows: {len(combined_df):,}")
    print(f"   Total columns: {len(combined_df.columns)}")
    print(f"   Symbols: {combined_df['symbol'].nunique()}")

    # Show statistics for key cross-sectional features
    cs_feature_groups = {
        'Volatility': ['vol_regime_rel'],
        'Relative Strength vs SPY': ['rel_strength_spy_norm', 'rel_strength_spy_slope20'],
        'Relative Strength vs Sector': ['rel_strength_sector_norm', 'rel_strength_sector_slope20'],
        'Relative Strength vs Subsector': ['rel_strength_subsector_norm', 'rel_strength_subsector_slope20'],
        'Cross-Sectional Momentum': ['xsec_mom_20d_z', 'xsec_mom_60d_z']
    }

    for group_name, features in cs_feature_groups.items():
        print(f"\n{'='*60}")
        print(f"📊 {group_name} Features")
        print(f"{'='*60}")
        
        for feature in features:
            if feature in combined_df.columns:
                data = combined_df[feature].dropna()
                if len(data) > 0:
                    print(f"\n{feature}:")
                    print(f"  Count: {len(data):,}")
                    print(f"  Mean:  {data.mean():.4f}")
                    print(f"  Std:   {data.std():.4f}")
                    print(f"  Min:   {data.min():.4f}")
                    print(f"  25%:   {data.quantile(0.25):.4f}")
                    print(f"  50%:   {data.median():.4f}")
                    print(f"  75%:   {data.quantile(0.75):.4f}")
                    print(f"  Max:   {data.max():.4f}")
                else:
                    print(f"\n{feature}: No data available")
            else:
                print(f"\n{feature}: Feature not found in data")

print(f"\n{'='*60}")
print("✅ Feature computation and visualization complete!")

## Summary

This notebook demonstrated:

### Single-Stock Features (`compute_single_stock_features`)
Features computed independently for each stock:
- Trend features (MA slopes, agreement)
- RSI indicators (14, 21, 30 periods)
- MACD features (histogram, derivative)
- Volatility regime (individual stock)
- Distance-to-MA features
- Range/breakout features (ATR)
- Volume features and shocks

### Cross-Sectional Features (`compute_cross_sectional_features`)
Features requiring comparisons across multiple stocks:
- Cross-sectional volatility context (relative to market median)
- Alpha-momentum (beta-adjusted returns)
- **Relative strength (vs SPY, sectors, subsectors)** - Requires SPY in the data!
- Market breadth indicators
- Cross-sectional momentum rankings (percentile ranks)

**Important:** For cross-sectional features to work properly, you must include benchmark ETFs (especially SPY) in the indicators_by_symbol dictionary!